# BERT Melon

## Project Imports

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [2]:
!pip install bert-tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Loading the dataset

In [5]:
df = pd.read_csv('data/agr_en_tw_gold.csv', names=['serial_no', 'tweets', 'sentiment'])
print(df.shape)
df.head(15)

(1257, 3)


,serial_no,tweets,sentiment
0,sme_1,6-0 hahahahahaha har ek pakistani ko yeh pic d...,NAG
1,sme_2,@sardanarohit :While entire nation is praying ...,OAG
2,sme_3,shoaib: wahab riaz looks like johnson. kapil d...,NAG
3,sme_4,"ramraj cottons r. ashwin southindian , chennai...",NAG
4,sme_5,#indvsuae jeet gaye... :) :) cmon starsports n...,NAG
5,sme_6,tt india 08:24 1.#indvspak 2.#mustwatchmsg 3.#...,NAG
6,sme_7,#shutdownjnu over democratic setup feeds these...,CAG
7,sme_8,umesh strikes double !!! uae 102/10 #bleedblue...,NAG
8,sme_9,bwahahahaha my last rt!!! 😂😂😂😂😂😂😂😂😂 #indvsuae,NAG
9,sme_10,Is JNU - Jihadi Nurturing University! #ShutDow...,OAG


In [6]:
df['sentiment'].unique()

array(['NAG', 'OAG', 'CAG'], dtype=object)

The aggression is categorized into different classes which are as follows - 

1. Aggression: It is a human behavior intended to harm another by verbally, physically and
psychologically. Overtly Aggressive (OAG): This class includes the following cases.
(a) Aggression shown openly with verbal attack directly pointed towards any group or
individuals.
(b) Attack commenced using abusive words or calling names or comparing in a derogatory
manner.
(c) By supporting false attack or supporting others comment.
(d) Sometimes these texts also contain indirect references.
2. Covertly Aggressive (CAG): In these attacks aggression is generally hidden and contains
sarcastic negative emotions due to its indirect nature. It can be summarized as follows.
(a) By using metaphorical words to attack an individual,nation,religion.
(b) Praising someone by criticizing group irrespective of being right or wrong.
(c) Sometimes these texts also contain direct references.
3. Non Aggressive (NAG): These statements generally lack the intention to be aggressive
and mostly used while referring to the correct facts, wishing or supporting individuals or
groups on social issues.

In [7]:
integer_mapping = {'NAG' : 0, 'OAG' : 1, 'CAG' : 2}
df = df.replace({'sentiment' : integer_mapping})

In [8]:
df['sentiment'].unique()

array([0, 1, 2])

In [9]:
train, test = train_test_split(df, test_size=0.2, random_state=42)

## Data Preprocessing

In [10]:
DATA_COLUMN = 'tweets'
LABEL_COLUMN = 'sentiment'
label_list = df['sentiment'].unique() # Use the InputExample class from BERT's run_classifier code to create examples from the data
print(label_list)

[0 1 2]


In [11]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [12]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"


def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(
            signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                  tokenization_info["do_lower_case"]])

    return bert.tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=do_lower_case)


tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
tokenizer.tokenize(train['tweets'].iloc[0])

['mis',
 '##bah',
 'ka',
 '##pr',
 '##aan',
 'hai',
 'jo',
 'ab',
 '##hi',
 'tak',
 'century',
 'nah',
 '##i',
 'l',
 '##ga',
 'pay',
 '##a',
 'ha',
 '##ha',
 '#',
 'ind',
 '##win',
 '##s',
 '#',
 'ind',
 '##vs',
 '##pa',
 '##k']

In [14]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 60
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 1005


INFO:tensorflow:Writing example 0 of 1005


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


## BERT Model

In [16]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(
        BERT_MODEL_HUB,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature="tokens",
        as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(
            tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)


In [17]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)

        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics.
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                true_pos = tf.metrics.true_positives(
                    label_ids,
                    predicted_labels)
                true_neg = tf.metrics.true_negatives(
                    label_ids,
                    predicted_labels)
                false_pos = tf.metrics.false_positives(
                    label_ids,
                    predicted_labels)
                false_neg = tf.metrics.false_negatives(
                    label_ids,
                    predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                    "true_positives": true_pos,
                    "true_negatives": true_neg,
                    "false_positives": false_pos,
                    "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                                                  loss=loss,
                                                  eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [18]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [19]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [20]:
OUTPUT_DIR = 'model/'

In [21]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [22]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f71fb1815c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f71fb1815c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [24]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 94 into model/model.ckpt.


INFO:tensorflow:Saving checkpoints for 94 into model/model.ckpt.


INFO:tensorflow:Loss for final step: 0.48558074.


INFO:tensorflow:Loss for final step: 0.48558074.


Training took time  0:06:52.361101


In [25]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [26]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/home/himmeldreng/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 94: eval_accuracy = 0.6944444, false_negatives = 11.0, false_positives = 11.0, global_step = 94, loss = 0.5937655, true_negatives = 90.0, true_positives = 140.0


INFO:tensorflow:Saving dict for global step 94: eval_accuracy = 0.6944444, false_negatives = 11.0, false_positives = 11.0, global_step = 94, loss = 0.5937655, true_negatives = 90.0, true_positives = 140.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 94: model/model.ckpt-94


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 94: model/model.ckpt-94


{'eval_accuracy': 0.6944444,
 'false_negatives': 11.0,
 'false_positives': 11.0,
 'loss': 0.5937655,
 'true_negatives': 90.0,
 'true_positives': 140.0,
 'global_step': 94}

In [27]:
def getPrediction(in_sentences):
    labels = ['NAG', 'OAG', 'CAG']
    input_examples = [run_classifier.InputExample(
        guid="", text_a=x, text_b=None, label=0) for x in in_sentences]  # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(
        input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(
        features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]


In [28]:
pred_sentences = list(test['tweets'])
type(pred_sentences)

list

In [29]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 252


INFO:tensorflow:Writing example 0 of 252


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from model/model.ckpt-94


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [30]:
predictions

[('jab sunday ko team india se pakistan takrayega tab stadium me sirf tringa laherayega #cwc15 #indvspak #worldcup http://t.co/dfhwom137b',
  array([-0.07745104, -4.3284955 , -2.7913182 ], dtype=float32),
  'NAG'),
 ('@narendramodi #ShutDownJNU we need education but not against nation',
  array([-4.5315466 , -0.57805234, -0.8480573 ], dtype=float32),
  'OAG'),
 ('#maukepechauka! india vs uae - india wins now turn for uae mauka mauka starsport ads. #indvssa, #indvsuae, #cwc15 http://t.co/kaimvazi2z',
  array([-0.12864865, -4.393031  , -2.2223516 ], dtype=float32),
  'NAG'),
 ('#indvsuae bhubaneswar kumar is back and',
  array([-0.6535944, -2.6725497, -0.8897672], dtype=float32),
  'NAG'),
 ('#ShutDownJNU JNU IS THE NURSERY OF ANTI-NATIONALS AND TERRORIST-LOVERS. KEEP IT TRENDING! LET THESE TRAITORS KNOW, THEIR GAME IS OVER.',
  array([-4.587813 , -0.3960981, -1.149219 ], dtype=float32),
  'OAG'),
 ('They are fasting unto death! #StandwithJNU #StandforJNU #WithyouJNU http:// fb.me/7Qx6LZ

In [31]:
x = np.array(predictions)
np.shape(x)

(252, 3)

In [32]:
x

array([['jab sunday ko team india se pakistan takrayega tab stadium me sirf tringa laherayega #cwc15 #indvspak #worldcup http://t.co/dfhwom137b',
        array([-0.07745104, -4.3284955 , -2.7913182 ], dtype=float32),
        'NAG'],
       ['@narendramodi #ShutDownJNU we need education but not against nation',
        array([-4.5315466 , -0.57805234, -0.8480573 ], dtype=float32),
        'OAG'],
       ['#maukepechauka! india vs uae - india wins now turn for uae mauka mauka starsport ads. #indvssa, #indvsuae, #cwc15 http://t.co/kaimvazi2z',
        array([-0.12864865, -4.393031  , -2.2223516 ], dtype=float32),
        'NAG'],
       ['#indvsuae bhubaneswar kumar is back and',
        array([-0.6535944, -2.6725497, -0.8897672], dtype=float32),
        'NAG'],
       ['#ShutDownJNU JNU IS THE NURSERY OF ANTI-NATIONALS AND TERRORIST-LOVERS. KEEP IT TRENDING! LET THESE TRAITORS KNOW, THEIR GAME IS OVER.',
        array([-4.587813 , -0.3960981, -1.149219 ], dtype=float32),
        'OAG'],
 

In [33]:
tweets = []
predicted_sentiment = []
for i in range(len(predictions)):
    tweets.append(predictions[i][0])
    predicted_sentiment.append(predictions[i][2])

In [37]:
output_dict = {'tweets' : tweets, 'predicted_sentiment' : predicted_sentiment}

In [39]:
!mkdir output

BERTMelon.ipynb  data  model  output  README.md


In [40]:
pd.DataFrame(output_dict).to_excel('output/output_bert.xlsx', index = False)